In [1]:
print("hola gordo")

hola gordo


In [1]:
import pandas as pd

In [49]:
PATH = r'C:\Users\brian\OneDrive\Escritorio\PROJECTS\UPV\sistemas_recomendacion\data'
genre_names = [
    'movie_id','unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 
    'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
    'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 
    'War', 'Western', 'movie_name']
df_items = pd.read_csv(PATH+'\items.txt',sep="	", encoding='latin-1',header=None, names= genre_names)
df_genre = pd.read_csv(PATH+'\genre.txt',sep="	", encoding='latin-1',header=None)
df_calif = pd.read_csv(PATH+r'\u1_base.txt',sep="	", encoding='latin-1',header=None,names=["users","movie_id","rating"])
df_users = pd.read_csv(PATH+r'\users.txt',sep="	", encoding='latin-1',header=None, names=["users","age","gender","ocuppation"])

In [57]:
df_concat = df_users.merge(df_calif, left_on='users', right_on='users')
df_concat.merge(df_items, left_on='movie_id', right_on='movie_id')

,users,age,gender,ocuppation,movie_id,rating,unknown,Action,Adventure,Animation,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movie_name
0,1,24,M,technician,1,5,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Toy Story (1995)
1,2,53,F,other,1,4,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Toy Story (1995)
2,6,42,M,executive,1,4,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Toy Story (1995)
3,13,47,M,educator,1,3,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Toy Story (1995)
4,16,21,M,entertainment,1,5,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Toy Story (1995)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,863,17,M,student,1679,3,0,0,0,0,...,0,0,0,0,1,0,1,0,0,B. Monkey (1998)
79996,863,17,M,student,1680,2,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Sliding Doors (1998)
79997,871,31,M,executive,907,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Vermin (1998)
79998,896,28,M,writer,1681,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,You So Crazy (1994)


In [ ]:
from lightgbm import LGBMClassifier


num_folds = 5
kf = KFold(n_splits = num_folds, random_state = 42)
error = 0
models = []
for i, (train_index, val_index) in enumerate(kf.split(train)):
    if i + 1 < num_folds:
        continue
    print(train_index.max(), val_index.min())
    train_X = train[train_index]
    val_X = train[val_index]
    train_y = target[train_index]
    val_y = target[val_index]
    lgb_train = lgb.Dataset(train_X, train_y > 0)
    lgb_eval = lgb.Dataset(val_X, val_y > 0)
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'learning_rate': 0.1,
            'feature_fraction': 0.8,
            'bagging_fraction': 1.0,
            'bagging_freq' : 0
             
            }
    gbm_class = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
               early_stopping_rounds=20,
               verbose_eval = 20)
    
    lgb_train = lgb.Dataset(train_X[train_y > 0], train_y[train_y > 0])
    lgb_eval = lgb.Dataset(val_X[val_y > 0] , val_y[val_y > 0])
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'learning_rate': 0.1,
            'feature_fraction': 0.8,
            'bagging_fraction': 1.0,
            'bagging_freq' : 0
                         }
    gbm_regress = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
               early_stopping_rounds=20,
               verbose_eval = 20)
#     models.append(gbm)

    y_pred = (gbm_class.predict(val_X, num_iteration=gbm_class.best_iteration) > .5) *\
    (gbm_regress.predict(val_X, num_iteration=gbm_regress.best_iteration))
    error += np.sqrt(mean_squared_error(y_pred, (val_y)))/num_folds
    print(np.sqrt(mean_squared_error(y_pred, (val_y))))
    break
print(error)

In [51]:
df_concatenado = pd.concat([df_users.set_index('users'), df_calif.set_index('users')], axis=1)
df_concateated = pd.concat([df_concatenado.set_index('movie_id'), df_items.set_index('movie_id')], axis=1)
# Si quieres resetear el índice después de la concatenación:
df_concatenado.reset_index(inplace=True)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [45]:
# Specify the columns to encode
columns_to_encode = ['gender','ocuppation']

# Perform one-hot encoding on selected columns
df_encoded = pd.get_dummies(df_concatenado
, columns=columns_to_encode)

In [46]:
df_encoded

,level_0,index,users,age,movie_id,rating,gender_F,gender_M,ocuppation_administrator,ocuppation_artist,...,ocuppation_marketing,ocuppation_none,ocuppation_other,ocuppation_programmer,ocuppation_retired,ocuppation_salesman,ocuppation_scientist,ocuppation_student,ocuppation_technician,ocuppation_writer
0,0,0,1,24,1,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,1,1,24,2,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,2,1,24,3,4,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,3,1,24,4,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4,4,1,24,5,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,79995,79995,943,22,1067,2,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
79996,79996,79996,943,22,1074,4,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
79997,79997,79997,943,22,1188,3,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
79998,79998,79998,943,22,1228,3,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
